# Importing

In [ ]:
import pandas as pd
# https://www.zillow.com/research/data/
# Credit: Zillow - ZHVI All Homes (SFR, Condo/Co-op) Time Series, Smoothed, Seasonally Adjusted
ZHVI_full = pd.read_csv("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
ZHVI_skimmed = ZHVI_full.drop(columns=['RegionID', 'SizeRank', 'RegionType', 'StateName'])

ZHVI_long = ZHVI_skimmed.melt(id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='Date', value_name='ZHVI')

ZHVI_long['Date'] = pd.to_datetime(ZHVI_long['Date'])
ZHVI_long['Date'] = ZHVI_long['Date'].dt.strftime('%Y-%m') # We dont need days
ZHVI_long['Date'] = pd.to_datetime(ZHVI_long['Date'])
ZHVI_df = ZHVI_long.dropna(subset=['ZHVI'])

# ^ Use that :)

In [ ]:
import pandas as pd
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "United States": "US"
}
# https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html
# Credit: US Census Bureau - Historical Population Change Data (1910-2020)
pop = pd.read_csv("apportionment.csv")
pop = pop.iloc[:,[0, 2, 3]]

pop["Name"] = pop["Name"].map(us_state_to_abbrev)
pop = pop.dropna()

state_pop = pop[pop["Name"] != "US"]
us_pop = pop[pop["Name"] == "US"]

# ^ Pops need to be interpolated

## Use this to create maps

You'll have to do a few things first.

To find your GeoJSON(map file):
1. Go to "https://hub.arcgis.com/datasets/esri::usa-zip-code-boundaries/explore"
2. Click the filter icon on the vertical small blue bar to your left
3. Select ZIP Code and State Abvreviation
4. Filter your state abreviation
5. Click the download tab below the filter
6. *SELECT TOGGLE FILTERS*
7. Download GeoJson :]
8. Please rename the file to something more specific, its the same no matter your filter and gets confusing

Next get your coordinates!
1. Go to google maps and hover over your state
2. go to the url and find the 2 numbers in it after the @  
    -> https://www.google.com/maps/@***28.0929856***,***-82.2101772***,7.25z?entry=ttu
3. Copy them and put them in the coordinates variables
4. Also select your state abbreviation, all caps

You will have run the code on your own machine, outside of the shared file, unless you want to upload to something like github and use this snippet to read it:

```python
# I think it has to be the raw path, cant just copy from ur browser theres a button, dont quote me on it though
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/fl_florida_zip_codes_geo.min.json') as response:
    State_geojson = json.load(response)
```

In [ ]:
import plotly.express as px
import pandas as pd
import json
import folium
import math

#Load the boundaries GeoJson
with open("C:\\Users\\gacan\\Downloads\\USA_ZIP_Code_Boundaries.geojson") as f:
    Zip_GeoJson = json.load(f)

#Specify state
#Choosing a date range
chosen_state = 'FL' # ALL CAPS - if you want multiple states thats easy to change
chosen_date = "2020-01" # YYYY-MM
df = ZHVI_df[ZHVI_df["State"] == chosen_state][["ZHVI","RegionName","Date"]]
df = df[df["Date"] == pd.to_datetime(chosen_date)][["RegionName","ZHVI"]]
df['ZHVI'] = df['ZHVI'].apply(lambda x: math.log(x))

In [ ]:
#Creates the Map!
coord1 = 28.0929856 #idrk if its NES or W
coord2 = -82.2101772
m = folium.Map(location=[coord1, coord2], zoom_start=7) # Larger states 7-8, smaller states 9-11
folium.Choropleth(
    geo_data= Zip_GeoJson,
    name='chloropleth',
    data=df,
    columns=["RegionName","ZHVI"],
    key_on='properties.ZIP_CODE', #<- GeoJson zip code key
    fill_color="RdYlBu",
    nan_fill_color="purple",
    nan_fill_opacity=0.7,
    fill_opacity=0.7,
    highlight=True,

).add_to(m)
""" #Going to work on this later - will be tooltip when hovering
#To Make it faster, remove from:
#Here ----------------------------------
def style_function(x):
    return {
        'color': 'black',
        'fillOpacity': 0,
        'weight': 1
        #There is a fillcolor option if u want
    }

tooltip = folium.GeoJsonTooltip(fields=['RegionName'], aliases=['ZHVI'], labels=True, sticky=False)

# Add GeoJson layer with style and highlight functions
folium.GeoJson(
    data=df,
    tooltip=tooltip,
    style_function=style_function,
).add_to(m)
# To Here ----------------------------------
"""
folium.LayerControl().add_to(m)
m

#The colors are actually working, everybodys poor except for a handfull of zips

# PROJECT 2 DAT301